In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import fbeta_score, precision_recall_curve, confusion_matrix

In [2]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [3]:
def threshold_by_fbeta(y_test: pd.Series, y_pred: list, *, beta: int = 1) -> tuple:
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    fbeta = ((1 + beta**2) * precision * recall) / (beta**2 * precision + recall)
    index = np.argmax(fbeta)
    return thresholds[index], fbeta[index]

In [4]:
df = pd.read_csv("/content/churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [6]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [7]:
final_transformers = []

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', MinMaxScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [8]:
feats = FeatureUnion(final_transformers)

In [9]:
classifiers = [
    RandomForestClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    LogisticRegression(random_state=42)
]

In [10]:
pipelines = []
metrics = {'threshold': [], 'fbeta': []}

In [11]:
for clf in classifiers:
    pipeline = Pipeline([
        ('features', feats),
        ('classifier', clf),
    ])
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict_proba(X_test)[:, 1]
    threshold, fbeta_score = threshold_by_fbeta(y_test, y_pred, beta=2)
    
    metrics['threshold'].append(threshold)
    metrics['fbeta'].append(fbeta_score)
    pipelines.append(pipeline)

In [12]:
metrics_df = pd.DataFrame(metrics, index=[clf.__class__.__name__ for clf in classifiers])
metrics_df.sort_values('fbeta', ascending=False)

,threshold,fbeta
GradientBoostingClassifier,0.154589,0.717822
RandomForestClassifier,0.150000,0.693304
LogisticRegression,0.135475,0.636624


In [13]:
best_model = pipelines[1]

In [14]:
y_pred_best = best_model.predict_proba(X_test)[:, 1]
_, _, thresholds = precision_recall_curve(y_test, y_pred)

In [15]:
def economic_metrics(y_pred_probas: list, thresholds: list) -> pd.DataFrame:
    profits = []
    expenses = []
    for thr in thresholds:
        preds = (y_pred_best >= thr).astype(bool)
        conf = confusion_matrix(y_test, preds)
        TP = conf[1][1]
        FP = conf[0][1]
        profits.append(TP * 2)
        expenses.append(FP * -1)

    results_df = pd.DataFrame({'threshold': thresholds, 'profit': profits, 'expense': expenses})
    results_df['total'] = results_df['profit'] + results_df['expense']
    return results_df

In [16]:
results = economic_metrics(y_pred_best, thresholds)

In [17]:
results.sort_values('total', ascending=False)

,threshold,profit,expense,total
2178,0.407250,608,-128,480
2175,0.406141,608,-128,480
2179,0.407268,608,-128,480
2180,0.407311,608,-128,480
2177,0.406888,608,-128,480
...,...,...,...,...
4,0.014457,1014,-1960,-946
3,0.014286,1014,-1963,-949
2,0.013231,1016,-1983,-967
1,0.011778,1018,-1989,-971


In [18]:
params = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__max_depth': [3, 4, 5]
}

In [19]:
grid = GridSearchCV(best_model,
                    param_grid=params,
                    cv=5,
                    refit=False)

In [20]:
search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': 5,
 'classifier__min_samples_leaf': 3,
 'classifier__n_estimators': 100}

In [21]:
optimal_model = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(n_estimators=100, max_depth=5, min_samples_leaf=3, random_state=42)),
])

In [22]:
optimal_model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [23]:
y_pred_optim = optimal_model.predict_proba(X_test)[:, 1]    

In [24]:
threshold, fbeta_score = threshold_by_fbeta(y_test, y_pred_optim, beta=2)

In [25]:
threshold, fbeta_score

(0.12823119166694474, 0.7170418006430869)

In [26]:
_, _, thresholds_optim = precision_recall_curve(y_test, y_pred_optim)

In [27]:
results_optim = economic_metrics(y_pred_best, thresholds_optim)

In [28]:
results_optim.sort_values('total', ascending=False)

,threshold,profit,expense,total
2020,0.404927,608,-128,480
2021,0.404966,608,-128,480
2022,0.406520,608,-128,480
2023,0.407940,608,-128,480
2024,0.409099,606,-127,479
...,...,...,...,...
1,0.009232,1018,-1991,-973
6,0.010006,1018,-1991,-973
7,0.010161,1018,-1991,-973
8,0.010165,1018,-1991,-973
